<h1>CS210 Project - File 2</h1>
Final re-arrangements of dataframes and feature-engineering before statistical analysis & machine learning

In [1]:
import pandas as pd
import numpy as np
import reverse_geocoder as rg

In [2]:
starbucks = pd.read_pickle('./pickles/montgomeryStarbucksStores.pickle')
emergency = pd.read_pickle('./pickles/emergencyCalls.pickle')

We will normalize our data to population, for this I've gathered the population data for the townships in Montgomery County.

In [3]:
# 2010 census was used to gather the following data
populations = {
    "Bala-Cynwyd": 9619, "King of Prussia": 19936,
    "Montgomeryville": 12624, "Plymouth Meeting": 6177,
    "Willow Grove": 15726, "Rockledge": 24998,
    "Flourtown": 4538, "Ardmore": 24486,
    "Horsham": 14842, "Wyncote": 3044,
    "Trappe": 3516, "Bridgeport": 4558,
    "Pottstown": 22392, "Trooper": 5744,
    "Narberth": 4284, "Norristown": 34347,
    "Glenside": 8384, "Jenkintown": 4426,
    "Conshohocken": 7842, "Wyndmoor": 5498,
    "Spring House": 3804, "Audubon": 8814,
    "Dresher": 5395, "Bryn Mawr": 3779,
    "Lansdale": 16282, "Sanatoga": 8378,
    "Collegeville": 5094
    }

Build a new dataframe consisting of the following:<ul>
            <li>Township Name</li>
            <li>Township Population</li>
            <li>Number of people per store living in the township</li>
            <li>Number of 911 calls received in the township</li>
        </ul>

To do so, first we will obtain the name of the township of locations of each store

In [12]:
search_results = rg.search(list(zip(list(starbucks.Latitude), list(starbucks.Longitude))))
sbTowns = pd.DataFrame([x['name'] for x in search_results], columns=['Town'])
sbTowns.head()

,Town
0,Rockledge
1,Willow Grove
2,Ardmore
3,Bala-Cynwyd
4,Bryn Mawr


We will do the same for the 911 data; however we don't have zip codes for this data set, instead what we have is the post code and that is not useful for the reverse geocoding library we're using. We will clean the data further

In [13]:
em = emergency.drop('zip', axis='columns')
em = em.drop('timeStamp', axis='columns')
em.head()

,lat,lng,title
0,40.297876,-75.581294,EMS: BACK PAINS/INJURY
1,40.258061,-75.264680,EMS: DIABETIC EMERGENCY
2,40.121182,-75.351975,Fire: GAS-ODOR/LEAK
3,40.116153,-75.343513,EMS: CARDIAC EMERGENCY
5,40.253473,-75.283245,EMS: HEAD INJURY


The reverse geocoder works in a feasible time if we provide the lat/long pairs in advance. Thus we do the following

In [14]:
# 1 - Obtain the reverse geocoding results
coordinates = zip(list(em.lat), list(em.lng))
reverseGeocodingOutput = rg.search(list(coordinates))

In [15]:
# 2 - Obtain the indexes of rows for which the call originates from within the Montgomery County
montgomery_indexes = []
town_list = list(sbTowns.Town)
for i in range(len(reverseGeocodingOutput)):
    if reverseGeocodingOutput[i]['name'] in town_list:
        montgomery_indexes.append(i)

In [16]:
# 3 - Select the rows we have just distinguished from the 911 calls data frame
filtered_rows = []
for i in montgomery_indexes:
    filtered_rows.append(list(em.iloc[i]))
filtered_em = pd.DataFrame(filtered_rows)
filtered_em.reset_index()
filtered_em.columns = ['Lat', 'Lng', 'Title']
filtered_em.head(3)

,Lat,Lng,Title
0,40.258061,-75.264680,EMS: DIABETIC EMERGENCY
1,40.121182,-75.351975,Fire: GAS-ODOR/LEAK
2,40.116153,-75.343513,EMS: CARDIAC EMERGENCY


We know that the rows in the filtered_em data frame has Starbucks stores. We need to get a data frame consisting of the town name instead of lat-long pairs.

In [17]:
coordinates = list(zip(list(filtered_em.Lat), list(filtered_em.Lng)))
em_search = rg.search(coordinates)
em_with_town = filtered_em.drop(filtered_em.columns[[0, 1]], axis="columns") # drop Lat & Lng columns

In [18]:
em_with_town['Town'] = [x['name'] for x in em_search]

In [20]:
em_with_town.head(3)

,Title,Town
0,EMS: DIABETIC EMERGENCY,Montgomeryville
1,Fire: GAS-ODOR/LEAK,Norristown
2,EMS: CARDIAC EMERGENCY,Norristown


In [21]:
sbTowns = sbTowns.Town.value_counts().reset_index()
sbTowns.columns = ['Town', 'Count']

In [22]:
sbTowns.head()

,Town,Count
0,Bala-Cynwyd,7
1,King of Prussia,6
2,Montgomeryville,5
3,Rockledge,3
4,Willow Grove,3


We've built the necessary data frames, now we need to perform merge operation

In [23]:
df = pd.merge(sbTowns, em_with_town, how="inner", on=['Town'])
df.columns=['Town', 'SBcount', 'Incident']
df.head()

,Town,SBcount,Incident
0,Bala-Cynwyd,7,Traffic: VEHICLE ACCIDENT -
1,Bala-Cynwyd,7,Traffic: DISABLED VEHICLE -
2,Bala-Cynwyd,7,Traffic: VEHICLE ACCIDENT -
3,Bala-Cynwyd,7,Traffic: VEHICLE ACCIDENT -
4,Bala-Cynwyd,7,EMS: FALL VICTIM


In [24]:
populationDF = pd.DataFrame(list(populations.keys()), list(populations.values())).reset_index()
populationDF.columns = ['Population', 'Town']
populationDF.head(3)

,Population,Town
0,9619,Bala-Cynwyd
1,19936,King of Prussia
2,12624,Montgomeryville


In [25]:
df = pd.merge(df, populationDF, how="inner", on=["Town"])

In [28]:
df.head(3)

,Town,SBcount,Incident,Population
0,Bala-Cynwyd,7,Traffic: VEHICLE ACCIDENT -,9619
1,Bala-Cynwyd,7,Traffic: DISABLED VEHICLE -,9619
2,Bala-Cynwyd,7,Traffic: VEHICLE ACCIDENT -,9619


In [29]:
df.tail(3)

,Town,SBcount,Incident,Population
96895,Audubon,1,Traffic: ROAD OBSTRUCTION -,8814
96896,Audubon,1,Traffic: ROAD OBSTRUCTION -,8814
96897,Audubon,1,EMS: UNCONSCIOUS SUBJECT,8814


As mentioned in file 1 and can be seen above, there are numerous types of incidents. We shall define a well defined categorical data group to ease our processing

In [30]:
def simplifyIncident(s):
    simplified = s[:s.find(':')]
    return simplified

In [31]:
df['Incident'] = df.apply(lambda x: simplifyIncident(x['Incident']), axis="columns");

In [32]:
df.head()

,Town,SBcount,Incident,Population
0,Bala-Cynwyd,7,Traffic,9619
1,Bala-Cynwyd,7,Traffic,9619
2,Bala-Cynwyd,7,Traffic,9619
3,Bala-Cynwyd,7,Traffic,9619
4,Bala-Cynwyd,7,EMS,9619


In [22]:
df.Incident.value_counts()

EMS        50209
Traffic    32522
Fire       14167
Name: Incident, dtype: int64

The data frame we want has the following columns:<br />
Town, SBcount, EMScount, TrafficCount, FireCount, Population<br />
We shall group our queries with respect to town and have one row for each town. For example, we can see the number of traffic related calls in Bala-Cynwyd

In [33]:
len(df[(df.Incident == "Traffic") & (df.Town == "Bala-Cynwyd")])

1430

In [34]:
ems_counts = []
fire_counts = []
traffic_counts = []

for town in populations.keys():
    ems_counts.append( len(df[(df.Incident == "EMS") & (df.Town == town)]) )
    fire_counts.append( len(df[(df.Incident == "Fire") & (df.Town == town)] ) )
    traffic_counts.append( len(df[(df.Incident == "Traffic") & (df.Town == town)] ) )

Now that we have these counts, we can drop the redundant rows in the dataframe to hold a single row per town

In [35]:
df = df.drop_duplicates(subset="Town").reset_index()

In [36]:
df = df.drop('index', axis="columns")

In [37]:
df = df.drop('Incident', axis="columns")

In [38]:
df['EMScount'] = ems_counts
df['FireCount'] = fire_counts
df['TrafficCount'] = traffic_counts
total_911_count = []
for i in range(27):
    total_911_count.append(ems_counts[i] + fire_counts[i] + traffic_counts[i])
df['TotalCalls'] = total_911_count

In [39]:
df

,Town,SBcount,Population,EMScount,FireCount,TrafficCount,TotalCalls
0,Bala-Cynwyd,7,9619,1109,473,1430,3012
1,King of Prussia,6,19936,2155,527,1193,3875
2,Montgomeryville,5,12624,1845,445,1741,4031
3,Rockledge,3,24998,2468,736,1955,5159
4,Willow Grove,3,15726,3209,794,2229,6232
5,Plymouth Meeting,3,6177,1668,550,1293,3511
6,Wyncote,2,3044,1054,369,803,2226
7,Flourtown,2,4538,1264,481,1093,2838
8,Pottstown,2,22392,1550,368,1525,3443
9,Bridgeport,2,4558,1336,378,1372,3086


We now have the data frame we want!

In [40]:
df.to_pickle('./pickles/finalDF_incident.pickle')